## Delhi Metro – Data Cleaning & Exploration Python Project

## Importing Necessary Libraries

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Let's load the dataset

In [10]:
df = pd.read_csv(r"C:\Users\Prerit\OneDrive\Desktop\Delhi-Metro-F\Delhi-Metro-Analysis-1\delhi_metro_raw.csv")

### Columns in the dataset

- `TripID`, `Date`, `From_Station`, `To_Station`, `Distance_km`, `Fare`, `Cost_per_passenger`, `Passengers`, `Ticket_Type`, `Remarks`

Note: our dataset contains:
- ~1% missing values in `Passengers` (NaN)
- ~1% 'NA' strings in `Ticket_Type`
- ~1% blank `Remarks` entries
- ~30% `From_Station` values with leading/trailing spaces and inconsistent casing (needs trimming)

In [ ]:
# Load dataset
csv_path = r'C:\prerit\Hungry-Coders\Delhi-Metro-Project\delhi_metro_updated.csv'
df = pd.read_csv(csv_path)
df.head()

In [ ]:
# Guys Let's Understand the data first
df.info()

In [ ]:
print(df["Ticket_Type"])
print(round(df["Fare"].sum(),2))

In [ ]:
df['From_st']

In [ ]:
print(df["Ticket_Type"].value_counts())


In [ ]:
mask = (df.index < 90000) & (df["Ticket_Type"].astype(str).str.strip().str.lower() == "return")
df.loc[mask, "Ticket_Type"] = "Tourist Card"



In [ ]:
print(df["Ticket_Type"].value_counts())
filtered = df[df['Ticket_Type'] == 'return']
print(filtered)

In [ ]:
sns.countplot(x="Ticket_Type", data=df)


## Next Step

Lets Undertand Frequency of data in column -"Ticket Type"

| Code Line                         | Meaning                                                         |
| ---------------------------- | --------------------------------------------------------------- |
| `value_counts(dropna=False)` | Bhai, ticket types kitni baar aaye, NA bhi batao!               |
| `== ""`                      | Kitni baar remarks me kuch likhna bhool gaya?                         |
| `isna().sum()`               | Passenger wala column me kitni missing (NaN) hain, batao jaldi! |


In [ ]:
# Show counts of special values- kyuki data clean bhi to karna padega tabhi to badhiya charts plot krenge 

print('\nTicket_Type value counts (including NA):')  #heading hai ye thik hai

print(df['Ticket_Type'].value_counts(dropna=False).head())

## Next step
ab remarks column mei jo blank values hai use check krte hai , thik hai ?

acha ruko , Passengers column bhi sath mei hi check kr lete hai ?

In [ ]:
print('\nRemarks blanks count: ', (df['Remarks'] == '').sum())  # bhai isse apne ko blank values kitna hai ye pta chal jayega

print('\nPassengers null count: ', df['Passengers'].isna().sum())  # and bhai isse apne ko NA kitan hai ye pta chal jayega


## 1) Trim whitespace and fix casing in `From_Station`

In [ ]:
# Trim whitespace and title-case stations
print('Before sample:')
print(df['From_Station'].sample(8))  #chalo random 8 data utha ke check krte hai, iss column mei kya issue hai ?


## Next step

chalo , isko clean kr dete hai, kyuki hame , "TRIM" , "upercase" ya "lowercase"  krna to ata hi hai , String jab padh rhe the tab bhi to sikha hi tha , hai na ?

In [ ]:
df['From_Station'] = df['From_Station'].astype(str).str.strip().str.title()

| Part                 | Meaning                                                                             |
| -------------------- | ----------------------------------------------------------------------------------- |
| `df['From_Station']` | Selecting the column                                                                |
| `astype(str)`        | Convert data to string (in case any value is non-string)                            |
| `str.strip()`        | Remove **leading & trailing whitespace** (e.g. `" Rajiv Chowk "` → `"Rajiv Chowk"`) |
| `str.title()`        | Capitalize **first letter of each word** (e.g. `"rajiv chowk"` → `"Rajiv Chowk"`)   |


Ab to syntax bhi samjh mei aagya ... ? life kitna asan hai na ?  chalo ab wapas check krte hai , data clean hua ya nhi 

## Next step

check krte hai wapas koi bhi random value nikal ke from the same column

In [ ]:
print('\nAfter sample:')
print(df['From_Station'].sample(8))

## 2) Handle 'NA' strings in `Ticket_Type` and missing entries

| Problem                                            | Example        |
| -------------------------------------------------- | -------------- |
| Dataset has `'NA'` as text, not real missing value | `'NA'` ≠ `NaN` |
| We want real missing values                        | `'NA'` → `NaN` |


In [ ]:
# Replace 'NA' string with actual NaN and then inspect
df['Ticket_Type'] = df['Ticket_Type'].replace('NA', np.nan)   # hn ji ham replace kr rhe hai "NA" ->"NAN"

print('Ticket_Type nulls:', df['Ticket_Type'].isna().sum())   # aur bas output check kr rhe hai 



## Next step

ab jab hame , total null values pta chal hi gya hai , so lets replace them 

Par ek min ruko , kya apko pta hai 

Mean -> Average (Sabhi values ko jod kar total number of values se divide karna)

Median -> Middle value (Data ko ascending ya descending order mein sort karne ke baad jo beech ki value hoti hai)

Mode -> Highest Frequency (Wo value jo data mein sabse zyaada baar aati hai)

In [ ]:
# Option: fill missing ticket types with 'Unknown' or the mode 
# matlab ye ki apne pass do option hai, ya to null ke jagah pe Unknown likh dete hai, ya phir , jo sabse jyda repeat hua hai wahi likh dete hai

mode_ticket = df['Ticket_Type'].mode(dropna=True)[0]
print('Mode ticket type:', mode_ticket)

# df['Ticket_Type'] = df['Ticket_Type'].fillna(mode_ticket)

## 3) Convert blank `Remarks` to NaN

In [ ]:
df['Remarks'] = df['Remarks'].replace('', np.nan)         # ye to simple hai, blank ko "NAN" se replace kr dete hai
print('Remarks nulls after conversion:', df['Remarks'].isna().sum())       #iss line se null ka count pta chal jayega

## 4) Handle missing `Passengers` values

Options:
- Impute with median or mean
- Drop rows (if only few)


In [ ]:
# Example: impute with median passengers
median_pass = int(df['Passengers'].median(skipna=True))
print('Median passengers:', median_pass)

## Next step

ab fill kr dete hai, jo median hamne calculate kiya hai , so we will replace null wiht median valuen now

In [ ]:
df['Passengers'] = df['Passengers'].fillna(median_pass)

print('Passengers nulls after replacing:', df['Passengers'].isna().sum())

## 5) Create `Profit` column

Profit per trip = (Fare - Cost_per_passenger) * Passengers

We'll create a `Profit` column and also a `Profit_per_passenger` for analysis.

In [ ]:
df['Profit_per_passenger'] = df['Fare'] - df['Cost_per_passenger']
# Profit could be negative for subsidized trips -> keep as is
df['Profit'] = df['Profit_per_passenger'] * df['Passengers']


print('Profit of Delhi metro total =>', round(df['Profit'].sum(), 2)) # it uses simple round method
print('Profit of Delhi metro total =>',np.round(df['Profit'].sum(), 2)) # it uses numpy round method

## Next step - Our first milestone

Lets see hamse kya badhiya kaam kiya hai

In [ ]:
#lets print individual column to see it clearly
print(df[['TripID','Fare','Cost_per_passenger','Passengers','Profit_per_passenger','Profit']].head())
print()
print(df.columns)
# lets also check total profit 
print('\nTotal profit across dataset:', df['Profit'].sum())

## 6) Save cleaned dataset (optional)

You can save the cleaned dataframe to a new CSV for further use.

In [ ]:
cleaned_path = 'delhi_metro_cleaned_Dataset.csv'
df.to_csv(cleaned_path, index=False)
print('Saved cleaned CSV to', cleaned_path)

## 7) Now lets Ploting charts to explore the data

Plots included:
1. Bar chart: top 10 stations by number of trips (From_Station)
2. Box plot: Fare distribution
3. Scatter plot: Distance vs Fare
4. Pie chart: Ticket_Type share
5. Histogram: Passengers distribution
6. Line chart: Total daily profit over time


In [ ]:
plt.figure(figsize=(10,5))
station_counts = df['From_Station'].value_counts().nlargest(10)
station_counts.plot(kind='bar')
plt.title('Top 10 From Stations by Trip Count')
plt.xlabel('From Station')
plt.ylabel('Trip Count')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
station_counts = df['From_Station'].value_counts().nlargest(10)

station_counts.plot(kind='barh')
plt.title('Top 10 From Stations by Trip Count')
plt.xlabel('Trip Count')
plt.ylabel('From Station')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
# Create mask for first 90k rows where From_Station matches either name
mask = (
    (df.index < 90000) &
    (
        df["From_Station"].astype(str).str.strip().str.lower().isin([
            "old delhi",
            "kirti nagar"
        ])
    )
)

# Update those rows
df.loc[mask, "From_Station"] = "Noida City Centre"

In [ ]:
print(df["From_Station"].value_counts().head(40))

## Total footfall

let calculate

In [ ]:
plt.figure(figsize=(10,6))
total_station_counts = (df['From_Station'].value_counts() + df['To_Station'].value_counts()).nlargest(8)

total_station_counts.plot(kind='barh')
plt.title('Top 10 Busiest Stations (From + To Counts)')
plt.xlabel('Total Usage Count')
plt.ylabel('Station')

plt.tight_layout()
plt.show()


### Top 5 From Stations by Trip Count with color gradient

In [ ]:
plt.figure(figsize=(10,6))
station_counts = df['From_Station'].value_counts().nlargest(5)

bars = station_counts.plot(kind='barh', cmap='viridis')
plt.title('Top 10 From Stations by Trip Count')
plt.xlabel('Trip Count')
plt.ylabel('From Station')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


### Top 10 From Stations by % Trip Contribution

In [ ]:
plt.figure(figsize=(10,6))
station_counts = df['From_Station'].value_counts().nlargest(10)
station_pct = (station_counts / station_counts.sum()) * 100

station_pct.plot(kind='barh')
plt.title('Top 10 From Stations by % Trip Contribution')
plt.xlabel('Percentage (%)')
plt.ylabel('From Station')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


### Plot all the above on a single canvas

In [ ]:
# Prepare data
from_counts = df['From_Station'].value_counts().nlargest(5)
from_pct = (from_counts / from_counts.sum()) * 100
total_counts = (df['From_Station'].value_counts() + df['To_Station'].value_counts()).nlargest(10)

plt.figure(figsize=(18, 14))

# Chart 1: Top 10 From Stations (Count)
plt.subplot(2, 2, 1)
from_counts.plot(kind='barh')
plt.title('Top 10 From Stations (Trip Count)')
plt.xlabel('Trip Count')
plt.ylabel('Station')
plt.gca().invert_yaxis()

# Chart 2: Percentage Contribution
plt.subplot(2, 2, 2)
from_pct.plot(kind='barh')
plt.title('Top 10 From Stations (% Contribution)')
plt.xlabel('Percentage (%)')
plt.ylabel('Station')
plt.gca().invert_yaxis()

# Chart 3: Total Station Usage (From + To)
plt.subplot(2, 2, 3)
total_counts.plot(kind='barh')
plt.title('Top 10 Busiest Stations (From + To)')
plt.xlabel('Total Trips')
plt.ylabel('Station')
plt.gca().invert_yaxis()

# Chart 4: Color Gradient Bar Chart for Ranking
plt.subplot(2, 2, 4)
sns.barplot(x=from_counts.values, y=from_counts.index, palette='viridis')
plt.title('Top Stations (Rank-based Color Gradient)')
plt.xlabel('Trip Count')
plt.ylabel('Station')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
plt.boxplot(df['Fare'].dropna(), vert=False)
plt.title('Fare Distribution (boxplot)')
plt.xlabel('Fare')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(df['Distance_km'], df['Fare'], alpha=0.3)
plt.title('Distance vs Fare')
plt.xlabel('Distance (km)')
plt.ylabel('Fare')
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
tt = df['Ticket_Type'].fillna('Unknown').value_counts()
plt.pie(tt, labels=tt.index, autopct='%1.1f%%')
plt.title('Ticket Type Share')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.hist(df['Passengers'], bins=30)
plt.title('Passengers Distribution')
plt.xlabel('Passengers per Trip')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
df['Date'] = pd.to_datetime(df['Date'])
daily = df.groupby('Date')['Profit'].sum().sort_index()
daily.rolling(window=7).mean().plot()
plt.title('7-day Rolling Average of Total Daily Profit')
plt.xlabel('Date')
plt.ylabel('Profit')
plt.show()

### Final notes

- The dataset is synthetic but realistic enough for practicing cleaning, imputation, trimming, and plotting.
- The notebook provides one example imputation (median). You can experiment with other strategies.
- Profit can be negative for trips where cost > fare, this is realistic for subsidized or low-ridership segments.


Keep practicing and exploring more and more projects
---
